# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
import nltk
nltk.download(['punkt', 'wordnet'])
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier
from sklearn.neural_network import MLPClassifier
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,HashingVectorizer,TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report,accuracy_score
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\savad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\savad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///cleanDatabase.db')
df = pd.read_sql_table('cleanData',engine)
df['related']=df['related'].replace(2,0)
X = df[['message']].values
Y = df.loc[:,'related':'direct_report'].values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    regex_express='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(regex_express, text)
    for url in detected_urls:
        text =text.replace(url,'url_link')
        
    regex_express='^[a-zA-Z0-9]'
    detected_spc = re.findall(regex_express, text)
    for spc in detected_spc:
        text =text.replace(spc,' ')
        
    tokens = nltk.tokenize.word_tokenize(text)
    lemmatizer =  nltk.stem.WordNetLemmatizer() 
    
    clean_tokens = []
    for tok in tokens:
        clean_tok =lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(BaggingClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.15, random_state=42)
pipeline.fit(X.ravel(), Y)

In [ ]:
y_pred=pipeline.predict(X_test.ravel())

In [ ]:
ypred=pd.DataFrame(y_pred,columns=df.loc[:,'related':'direct_report'].columns)
yreal=pd.DataFrame(Y_test,columns=df.loc[:,'related':'direct_report'].columns)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
for coll in ypred.columns:
    print('-----'+coll+'-----')
    print(classification_report(yreal[coll], ypred[coll]))
    print('Accuracy score : {}'.format(accuracy_score(yreal[coll], ypred[coll])))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'clf__estimator__max_features':[2,6,10,100]
}
cv = GridSearchCV(pipeline, parameters).fit(X_train.ravel(), Y_train)
print(cv.best_score_)    
print(cv.best_params_)  

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(BaggingClassifier(max_features=2)))
    ])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.15, random_state=42)
pipeline.fit(X.ravel(), Y)
y_pred=pipeline.predict(X_test.ravel())
ypred=pd.DataFrame(y_pred,columns=df.loc[:,'related':'direct_report'].columns)
yreal=pd.DataFrame(Y_test,columns=df.loc[:,'related':'direct_report'].columns)
for coll in ypred.columns:
    print('-----'+coll+'-----')
    print(classification_report(yreal[coll], ypred[coll]))
    print('Accuracy score : {}'.format(accuracy_score(yreal[coll], ypred[coll])))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
features=[HashingVectorizer(tokenizer=tokenize),TfidfVectorizer(tokenizer=tokenize)]
parameters = {
    'clf__estimator__learning_rate':[0.1,0.25,0.5,0.75,1],
    'clf__estimator__n_estimators':[2,5,8,10,20,30,50,100]
}

for faet in features:
    pipeline = Pipeline([
    ('vect', faet),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    cv = GridSearchCV(pipeline, parameters).fit(X_train.ravel(), Y_train)
    print(cv.best_score_)    
    print(cv.best_params_)

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(pipeline, open('model_to_use.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.